In [1]:
import pandas as pd
import numpy as np

# Load clean dataset
df = pd.read_csv('../cleaned_data/marketing_touchpoints_clean.csv')

# Convert date column to datetime
df['touchpoint_date'] = pd.to_datetime(df['touchpoint_date'])

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print("\nColumns available:")
print(df.columns.tolist())

Dataset loaded: 16205 rows, 44 columns

Columns available:
['touchpoint_id', 'account_id', 'company_name', 'industry', 'company_size', 'touchpoint_date', 'year', 'month', 'month_name', 'quarter', 'day_of_week', 'marketing_channel', 'campaign_name', 'campaign_type', 'touchpoint_position', 'ad_spend', 'impressions', 'clicks', 'click_through_rate', 'cost_per_click', 'landing_page_visits', 'form_submissions', 'demo_requests', 'mql_generated', 'sql_generated', 'opportunities_created', 'deal_closed', 'deal_value', 'sales_cycle_days', 'roi', 'roas', 'cost_per_acquisition', 'estimated_ltv', 'device_type', 'geographic_region', 'lead_source', 'engagement_score', 'engagement_level', 'email_opens', 'email_clicks', 'webinar_attendance', 'content_downloads', 'week_of_year', 'conversion_flag']


In [2]:
print("="*70)
print("CREATING DIM_DATE")
print("="*70)

# Extract unique dates and create date dimension
dim_date = df[['touchpoint_date', 'year', 'month', 'month_name', 'quarter', 
               'day_of_week', 'week_of_year']].drop_duplicates()

# Create date_key (format: YYYYMMDD)
dim_date['date_key'] = dim_date['touchpoint_date'].dt.strftime('%Y%m%d').astype(int)

# Rename for clarity
dim_date = dim_date.rename(columns={'touchpoint_date': 'full_date'})

# Reorder columns
dim_date = dim_date[['date_key', 'full_date', 'year', 'month', 'month_name', 
                     'quarter', 'day_of_week', 'week_of_year']]

# Sort by date
dim_date = dim_date.sort_values('date_key').reset_index(drop=True)

print(f"✓ Dim_Date created: {len(dim_date)} unique dates")
print(f"  Date range: {dim_date['full_date'].min()} to {dim_date['full_date'].max()}")
print("\nFirst 5 rows:")
print(dim_date.head())

CREATING DIM_DATE
✓ Dim_Date created: 366 unique dates
  Date range: 2024-01-01 00:00:00 to 2024-12-31 00:00:00

First 5 rows:
   date_key  full_date  year  month month_name  quarter day_of_week  \
0  20240101 2024-01-01  2024      1    January        1      Monday   
1  20240102 2024-01-02  2024      1    January        1     Tuesday   
2  20240103 2024-01-03  2024      1    January        1   Wednesday   
3  20240104 2024-01-04  2024      1    January        1    Thursday   
4  20240105 2024-01-05  2024      1    January        1      Friday   

   week_of_year  
0             1  
1             1  
2             1  
3             1  
4             1  


In [3]:
print("\n" + "="*70)
print("CREATING DIM_ACCOUNT")
print("="*70)

# Create account dimension - keep first occurrence of each account_id
# This handles cases where same account has varying attributes across touchpoints

dim_account = df[['account_id', 'company_name', 'industry', 'company_size', 
                  'geographic_region']].drop_duplicates(subset=['account_id'], keep='first')

# Sort by account_id
dim_account = dim_account.sort_values('account_id').reset_index(drop=True)

print(f"✓ Dim_Account created: {len(dim_account)} unique accounts")
print(f"✓ Each account_id appears exactly once")

# Verify no duplicates
duplicate_check = dim_account['account_id'].duplicated().sum()
print(f"✓ Duplicate account_ids: {duplicate_check}")

print("\nFirst 5 rows:")
print(dim_account.head())


CREATING DIM_ACCOUNT
✓ Dim_Account created: 900 unique accounts
✓ Each account_id appears exactly once
✓ Duplicate account_ids: 0

First 5 rows:
   account_id company_name       industry company_size geographic_region
0        1001  Company_360  Manufacturing   Mid-Market     North America
1        1002   Company_27          Media          Smb             Latam
2        1003  Company_160     Consulting          Smb              Apac
3        1004  Company_249        Finance          Smb              Apac
4        1005  Company_240        Unknown          Smb              Emea


In [4]:
print("\n" + "="*70)
print("CREATING DIM_CHANNEL")
print("="*70)

# Create channel dimension with unique channel combinations
dim_channel = df[['marketing_channel', 'touchpoint_position', 'device_type', 
                  'lead_source']].drop_duplicates()

# Create channel_key (sequential ID)
dim_channel['channel_key'] = range(1, len(dim_channel) + 1)

# Reorder columns
dim_channel = dim_channel[['channel_key', 'marketing_channel', 'touchpoint_position', 
                           'device_type', 'lead_source']]

# Sort by channel name
dim_channel = dim_channel.sort_values('marketing_channel').reset_index(drop=True)

print(f"✓ Dim_Channel created: {len(dim_channel)} unique channel combinations")
print("\nFirst 10 rows:")
print(dim_channel.head(10))


CREATING DIM_CHANNEL
✓ Dim_Channel created: 384 unique channel combinations

First 10 rows:
   channel_key  marketing_channel touchpoint_position device_type lead_source
0           86  Content Marketing         First Touch      Tablet    Outbound
1          145  Content Marketing          Last Touch      Mobile    Referral
2          331  Content Marketing          Last Touch     Desktop     Inbound
3           50  Content Marketing         First Touch     Desktop    Referral
4           52  Content Marketing         First Touch     Desktop     Inbound
5           53  Content Marketing          Last Touch     Desktop    Referral
6          309  Content Marketing         Multi Touch     Desktop    Referral
7           58  Content Marketing         Multi Touch      Mobile    Referral
8          300  Content Marketing         First Touch      Tablet     Inbound
9           63  Content Marketing         Multi Touch      Mobile     Inbound


In [5]:
print("\n" + "="*70)
print("CREATING DIM_CAMPAIGN")
print("="*70)

# Create campaign dimension with unique campaigns
dim_campaign = df[['campaign_name', 'campaign_type']].drop_duplicates()

# Create campaign_key (sequential ID)
dim_campaign['campaign_key'] = range(1, len(dim_campaign) + 1)

# Reorder columns
dim_campaign = dim_campaign[['campaign_key', 'campaign_name', 'campaign_type']]

# Sort by campaign name
dim_campaign = dim_campaign.sort_values('campaign_name').reset_index(drop=True)

print(f"✓ Dim_Campaign created: {len(dim_campaign)} unique campaigns")
print("\nAll campaigns:")
print(dim_campaign)


CREATING DIM_CAMPAIGN
✓ Dim_Campaign created: 45 unique campaigns

All campaigns:
    campaign_key          campaign_name  campaign_type
0             45   Brand_Awareness_2024        Unknown
1             29   Brand_Awareness_2024      Awareness
2             19   Brand_Awareness_2024      Retention
3              5   Brand_Awareness_2024     Conversion
4             12   Brand_Awareness_2024  Consideration
5             42      Fall_Content_Push        Unknown
6             18      Fall_Content_Push     Conversion
7             11      Fall_Content_Push      Awareness
8              1      Fall_Content_Push  Consideration
9              2      Fall_Content_Push      Retention
10            37    Holiday_Retargeting        Unknown
11            16    Holiday_Retargeting  Consideration
12            34    Holiday_Retargeting      Retention
13             6    Holiday_Retargeting      Awareness
14             4    Holiday_Retargeting     Conversion
15            10    Lead_Gen_Initiati

In [6]:
print("\n" + "="*70)
print("CREATING FACT_TOUCHPOINTS")
print("="*70)

# Start with copy of main dataset
fact_touchpoints = df.copy()

# Create date_key to match Dim_Date
fact_touchpoints['date_key'] = fact_touchpoints['touchpoint_date'].dt.strftime('%Y%m%d').astype(int)

# Merge to get channel_key
fact_touchpoints = fact_touchpoints.merge(
    dim_channel[['channel_key', 'marketing_channel', 'touchpoint_position', 'device_type', 'lead_source']],
    on=['marketing_channel', 'touchpoint_position', 'device_type', 'lead_source'],
    how='left'
)

# Merge to get campaign_key
fact_touchpoints = fact_touchpoints.merge(
    dim_campaign[['campaign_key', 'campaign_name', 'campaign_type']],
    on=['campaign_name', 'campaign_type'],
    how='left'
)

# Select only fact table columns (IDs + measures)
fact_columns = [
    'touchpoint_id', 'account_id', 'date_key', 'channel_key', 'campaign_key',
    'ad_spend', 'impressions', 'clicks', 'click_through_rate', 'cost_per_click',
    'landing_page_visits', 'form_submissions', 'demo_requests',
    'mql_generated', 'sql_generated', 'opportunities_created',
    'deal_closed', 'deal_value', 'sales_cycle_days',
    'roi', 'roas', 'cost_per_acquisition', 'estimated_ltv',
    'engagement_score', 'email_opens', 'email_clicks', 'content_downloads'
]

fact_touchpoints = fact_touchpoints[fact_columns]

# Sort by touchpoint_id
fact_touchpoints = fact_touchpoints.sort_values('touchpoint_id').reset_index(drop=True)

print(f"✓ Fact_Touchpoints created: {len(fact_touchpoints)} rows")
print(f"✓ Measures included: {len(fact_columns) - 5}")
print("\nFirst 5 rows:")
print(fact_touchpoints.head())


CREATING FACT_TOUCHPOINTS
✓ Fact_Touchpoints created: 16205 rows
✓ Measures included: 22

First 5 rows:
   touchpoint_id  account_id  date_key  channel_key  campaign_key  ad_spend  \
0          10001        1103  20240401          129            25   8491.73   
1          10003        1861  20240115          328             6   4213.33   
2          10004        1271  20240204          163             3  11274.85   
3          10005        1107  20240906           31            30   2353.42   
4          10006        1072  20240930           64            29   5474.42   

   impressions   clicks  click_through_rate  cost_per_click  ...  deal_value  \
0      36704.0   7441.0               20.71          416.25  ...         0.0   
1     413704.0      0.0                7.84          160.83  ...         0.0   
2     179932.0  13595.0                9.96          393.08  ...         0.0   
3      69781.0   9184.0                6.69          342.07  ...         0.0   
4     220559.0  1312

In [8]:
# Re-export corrected Dim_Account
dim_account.to_csv('../model/Dim_Account.csv', index=False)
print(f"✓ Dim_Account.csv UPDATED: {len(dim_account)} unique accounts, no duplicates")

✓ Dim_Account.csv UPDATED: 900 unique accounts, no duplicates


In [7]:
print("\n" + "="*70)
print("EXPORTING STAR SCHEMA TABLES")
print("="*70)

# Create model folder if it does not exist
import os
model_path = '../model'
if not os.path.exists(model_path):
    os.makedirs(model_path)
    print(f"✓ Created folder: {model_path}")

# Export dimension tables
dim_date.to_csv(f'{model_path}/Dim_Date.csv', index=False)
print(f"✓ Dim_Date.csv exported: {len(dim_date)} rows")

dim_account.to_csv(f'{model_path}/Dim_Account.csv', index=False)
print(f"✓ Dim_Account.csv exported: {len(dim_account)} rows")

dim_channel.to_csv(f'{model_path}/Dim_Channel.csv', index=False)
print(f"✓ Dim_Channel.csv exported: {len(dim_channel)} rows")

dim_campaign.to_csv(f'{model_path}/Dim_Campaign.csv', index=False)
print(f"✓ Dim_Campaign.csv exported: {len(dim_campaign)} rows")

# Export fact table
fact_touchpoints.to_csv(f'{model_path}/Fact_Touchpoints.csv', index=False)
print(f"✓ Fact_Touchpoints.csv exported: {len(fact_touchpoints)} rows")

print("\n" + "="*70)
print("STAR SCHEMA TABLES READY")
print("="*70)
print("\nFiles created in /model folder:")
print("  1. Dim_Date.csv")
print("  2. Dim_Account.csv")
print("  3. Dim_Channel.csv")
print("  4. Dim_Campaign.csv")
print("  5. Fact_Touchpoints.csv")


EXPORTING STAR SCHEMA TABLES
✓ Dim_Date.csv exported: 366 rows
✓ Dim_Account.csv exported: 900 rows
✓ Dim_Channel.csv exported: 384 rows
✓ Dim_Campaign.csv exported: 45 rows
✓ Fact_Touchpoints.csv exported: 16205 rows

STAR SCHEMA TABLES READY

Files created in /model folder:
  1. Dim_Date.csv
  2. Dim_Account.csv
  3. Dim_Channel.csv
  4. Dim_Campaign.csv
  5. Fact_Touchpoints.csv
